Explorando dados

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd 
import time

options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')

navegador = webdriver.Chrome()
navegador.get('https://www.iso.org/obp/ui/en/')
wait = WebDriverWait(navegador, 10)

filtro_div = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "v-select-optiongroup-xmltype")))
standards_input = filtro_div.find_element(By.XPATH, "//label[text()='Standards']/preceding-sibling::input")
navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")
standards_input.click()


search_box = navegador.find_element(By.XPATH, '//*[@id="obpui-105541713"]/div/div[2]/div/div/div[2]/div/div/div[2]/div/div/div/div/div/div[2]/div/div[2]/div/div[2]/div/div/div[2]/input')
search_box.send_keys('')
search_box.send_keys(Keys.RETURN)
wait

wait.until(EC.presence_of_element_located((By.CLASS_NAME, "v-label-std-title")))
time.sleep(60)

def scroll_and_click(element):
    navegador.execute_script("arguments[0].scrollIntoView();", element)
    time.sleep(1)
    element.click()
    wait.until(EC.staleness_of(navegador.find_element(By.XPATH, "//div[@class='v-label-std-ref'][last()]")))


def acha_titulos():
    titulos = {}
    while True:
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "v-label-std-title")))
        std_refs = navegador.find_elements(By.CLASS_NAME, "v-label-std-ref")
        std_titles = navegador.find_elements(By.CLASS_NAME, "v-label-std-title")
        for i in range(len(std_refs)):
            titulo = std_titles[i].text
            ref = std_refs[i].text
            titulos[ref] = titulo
            count += 1

        if count % 300 == 0:
            page_count += 1
            print(f"Collected {count} titles from {page_count} pages")
            if page_count > 1:
                print("Waiting for 10 seconds before continuing...")
                time.sleep(10)
        
        try:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "v-label-std-title")))
            next_button = navegador.find_element(By.XPATH, "//div[@class='v-button v-widget i-paging v-button-i-paging last v-button-last']")
            if next_button.get_attribute("class") == "v-button v-widget i-paging v-button-i-paging last v-button-last v-disabled":
                break
            scroll_and_click(next_button)
        except:
            break
            
    return titulos

titulos = acha_titulos()
print(titulos)

df = pd.DataFrame.from_dict(titulos, orient='index').reset_index().rename(columns={'index': 'chave', 0: 'descricao'})
df = df.explode('chave')
df = df.fillna('')
df[['ISO', 'VERSAO']] = df['chave'].str.split(':', expand=True)
df[['colunaex','IDIOMA']] = df['chave'].str.split('(', expand=True)
df['ISO'] = df['ISO'].str.split('(', n=1).str.get(0)
df['VERSAO'] = df['VERSAO'].str.split('(', n=1).str.get(0)
df['IDIOMA'] = df['IDIOMA'].str.split(')', n=1).str.get(0)
df = df.drop('chave', axis=1)
df = df.drop('colunaex', axis=1)    
df
df.to_csv('iso_stardards.csv', index=False, sep=";")


{'ISO 14705:2016(en)': 'Fine ceramics (advanced ceramics, advanced technical ceramics) — Test method for hardness of monolithic ceramics at room temperature', 'ISO 17905:2015(en)': 'Ships and marine technology — Installation, inspection and maintenance of container securing devices for ships', 'ISO 20519:2021(en)': 'Ships and marine technology — Specification for bunkering of liquefied natural gas fuelled vessels', 'ISO/TS 11869:2012(en)': 'Fermented milks — Determination of titratable acidity — Potentiometric method', 'ISO 9934-3:2015(en)': 'Non-destructive testing — Magnetic particle testing — Part 3: Equipment', 'ISO 16840-4:2009(en)': 'Wheelchair seating — Part 4: Seating systems for use in motor vehicles', 'ISO 4296-1:1984(en)': 'Manganese ores — Sampling — Part 1: Increment sampling', 'ISO/TS 18621-31:2020(en)': 'Graphic technology — Image quality evaluation methods for printed matter — Part 31: Evaluation of the perceived resolution of printing systems with the Contrast–Resoluti

In [ ]:
df = pd.DataFrame.from_dict(titulos, orient='index').reset_index().rename(columns={'index': 'chave', 0: 'descricao'})
df = df.explode('chave')
df = df.fillna('')
df[['ISO', 'VERSAO']] = df['chave'].str.split(':', expand=True)
df[['colunaex','IDIOMA']] = df['chave'].str.split('(', expand=True)
df['ISO'] = df['ISO'].str.split('(', n=1).str.get(0)
df['VERSAO'] = df['VERSAO'].str.split('(', n=1).str.get(0)
df['IDIOMA'] = df['IDIOMA'].str.split(')', n=1).str.get(0)
df = df.drop('chave', axis=1)
df = df.drop('colunaex', axis=1)
df
df.to_csv('iso.csv', index=False, sep=";")